In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

try:
    import tensorflow as tf
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
except ImportError:
    print("Installing required libraries")
    import subprocess
    subprocess.check_call(['pip', 'install', 'tensorflow', '--break-system-packages'])
    import tensorflow as tf
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau

try:
    import yfinance as yf
except ImportError:
    print("Installing yfinance")
    import subprocess
    subprocess.check_call(['pip', 'install', 'yfinance', '--break-system-packages'])
    import yfinance as yf